In [ ]:
import os
import csv
import json
from pathlib import Path
from tqdm import tqdm

In [ ]:
source = "/run/user/1000/gvfs/smb-share:server=10.50.1.4,share=datalakev1/05_labeling_company/1_detection_request/01_stage1/PFAI_stage1/"
json_path = source + "AQ1-PFAI-STAGE-1_472_2024-08-05.json"

In [ ]:
def read_json(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
    else:
        print("doesn't exist",file_path)

In [ ]:
json_data = read_json(json_path)

In [ ]:
#init variables
# cur_image_id = None
duplication_cnt=0

for entry in json_data[114]:
    filename = entry["image_id"]
    elements = entry["elements"]
    # save file_name
    source_name = Path(filename).stem
    file_name = source_name+".txt"

    # duplicated element check
    if(os.path.isfile(file_name)): 
        duplication_cnt += 1
        continue
        
    # for each bbox
    for elem in elements:
        geometry = elem["geometry"]
        class_name = elem["class"]

        if class_name.lower() == "tuna" : class_name=0
        else: continue

        # filter non bbox labels (dot and comments)
        if isinstance(geometry, dict): continue 
        
        # get points
        xyxy = geometry[0]
        pt1, pt2, pt3, pt4 = xyxy
        
        xmin = pt1["x"]
        ymax = pt1["y"]
        xmax = pt3["x"]
        ymin = pt3["y"]
        
        x_center = (xmax + xmin) / 2 
        y_center = (ymax + ymin) / 2
        width = xmax - xmin
        height = ymax - ymin

        # write to file
        if(os.path.isfile(file_name)):
            with open(file_name,'a+') as f:
                f.write(f'{class_name} {x_center} {y_center} {width} {height}\n')
        else:
            with open(file_name,'w') as f:
                f.write(f'{class_name} {x_center} {y_center} {width} {height}\n')    

print("[duplication_cnt] ",duplication_cnt)

In [ ]:
len(json_data[1])

In [ ]:
len(json_data)

In [ ]:
json_data[1]['image_id']

In [ ]:
len(json_data[100][1]['elements'])

In [ ]:
for i in range(14):
    print(100+i,len(json_data[100+i]))

In [ ]:
json_data[0]

In [ ]:
len(json_data[108])

In [ ]:
json_data[108]